In [ ]:
from data_loader import *
from __future__ import division
from __future__ import print_function
import pandas as pd
import bokeh
import numpy as np

import bokeh.plotting as bp
from bokeh.plotting import figure, output_file, show,gridplot
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models.axes import DatetimeAxis

from datetime import datetime
from datetime import timedelta

import pandas as pd

bp.output_notebook()

In [ ]:
data = DataLoader('E3_7E_44_77_F2_38.scn') #badge data from short experiment
#data = DataLoader('FF_6B_D4_CB_22_FA.scn') #badge data from short experiment

session_start_time = datetime(2016,1,27,14,26,0)
session_end_time = datetime(2016,1,27,14,46,0)

#session_start_time = datetime(2016,1,27,15,10,0)
#session_end_time = datetime(2016,1,27,16,5,0)

#data = DataLoader('badge_v33_EC_21_82_A8_0B_59.scn') #badge data from a 4 days test

In [ ]:
freq = data.data[0]['Frequency'] #assuming same freq. for all samples

In [ ]:
# take only samples
signals = map(lambda x: x['samples'], data.data)

# convert 
con_signals = [map(lambda x: {"datetime": x["time"], "signal": x['signal']}, i)  for i in signals]
#con_signals[0]

# Append all samples into an array
singals_all = []
for s in con_signals:
    singals_all.extend(s)
    
# sort
sdata = sorted(singals_all, key=lambda x: x['datetime'])

# Check for gaps in data (before loading into pandas. It's faster)

In [ ]:
def check_raw_gaps(arr, max_delta_sec = 15):
    #max_diff = np.timedelta64(max_delta_sec, 's')
    max_diff = timedelta(seconds=1);
    prev = None
    for i in range (0,len(arr)): # iterate over all lines
        #print df.iloc[i].datetime
        curr = arr[i]['datetime']
        if (prev):
            diff = curr-prev
            if diff > max_diff:
                print ("{},{},{}".format(curr,prev,diff))
                #break
        prev = curr

In [ ]:
check_raw_gaps(sdata,15)

# Loading data infor Pandas

In [ ]:
df = pd.DataFrame()

In [ ]:
sdata[10:20]

In [ ]:
# Append all samples into a DataFrame
#for s in sdata:
df = df.append(pd.DataFrame(sdata))

In [ ]:
df = df.set_index(pd.DatetimeIndex(df['datetime']))
#df.index

In [ ]:
df_fil = df.ix[session_start_time:session_end_time]

In [ ]:
df_fil[0:10]

In [ ]:
# calc a moving average
sr_mean = pd.rolling_mean(df_fil['signal'], window=4*60, min_periods=1, center=False) # 4 samples per sec, 60 seconds

# smooth signal
sr_smooth = pd.rolling_mean(df_fil['signal'], window=5, min_periods=1, center=False) # 5 samples

sr_mean[0:10]

In [ ]:
# calc RMS
smpls = 4 * 30;
#series_sq=df_fil['signal']*df_fil['signal'] # sqaure values
series_sq=sr_smooth*sr_smooth # sqaure values
series_sq_mean = pd.rolling_mean(series_sq, window=smpls, min_periods=1, center=False) # mean square
series_sq_rms = series_sq_mean.apply(np.sqrt)

In [ ]:
tools = "xwheel_zoom,xpan,reset,resize,save"
p = figure(title="Signal", x_axis_label='x', y_axis_label='y'
           ,x_axis_type="datetime",width=1000, height=400,y_range=[0, 20], tools=tools)
# add a line renderer with legend and line thickness
p.line(df_fil.index.to_pydatetime(), df_fil.signal, legend="Raw", line_width=2, color='blue')
p.line(sr_smooth.index,sr_smooth, legend="Smooth5", line_width=2, color='Red')
p.line(series_sq_rms.index,series_sq_rms, legend="RMS30", line_width=2, color='Gray')
p.line(sr_mean.index,sr_mean, legend="Mean60", line_width=2, color='green')
p.below[0].formatter.formats = dict(years=['%Y'],months=['%b %Y'],days=['%d %b %Y'],hours=["%I:%M %P"],minutes=["%I:%M %P"]) #minutes is the one i'm interested in

# show the results
show(p)

In [ ]:
#
#series_sq_rms
df_full = df_fil.copy()
df_full['smooth']=sr_smooth
df_full['rms']=series_sq_rms

# is speaking interval
df_full['is_speak'] = df_full.apply(lambda x : 1 if x['smooth'] > x['rms'] else 0, axis=1)

In [ ]:
df_full

In [ ]:
tools = "xwheel_zoom,xpan,reset,resize,save"
p1 = figure(title="Signal", x_axis_label='x', y_axis_label='y'
           ,x_axis_type="datetime",width=1000, height=400,y_range=[0, 20], tools=tools)
# add a line renderer with legend and line thickness
p1.line(df_full.index,df_full['is_speak'], legend="Speak", line_width=2, color='blue')
p1.line(df_full.index,df_full['smooth'], legend="Smooth5", line_width=2, color='Red')
p1.line(df_full.index,df_full['rms'], legend="RMS30", line_width=2, color='Gray')
p1.below[0].formatter.formats = dict(years=['%Y'],months=['%b %Y'],days=['%d %b %Y'],hours=["%I:%M %P"],minutes=["%I:%M %P"]) #minutes is the one i'm interested in

# show the results
show(p1)

In [ ]:
# remove short intervals, "fill in" short blanks
# expects a data frame that indicates start time of samples that include speak. The freq indicates the time
# between samples (so each sample actually represents a speaking interval which is freq ms long).
def determine_speakers_intervals(df_speaker, group_max_time_diff_ms = 1000, min_interval_time_ms = 300, freq_ms = 250):
        def groupwhile(df, fwhile):
            groups = []
            i = 0
            while i < len(df):
                j = i
                while j < len(df) - 1 and fwhile(i,j, j + 1):
                    j = j + 1
                groups.append(df.iloc[i:j + 1])
                i = j + 1
            return groups

        #group_max_time_diff_sec = max md between records from the same user
        #min_interval_time_ms = minimum ms duration of speaking interval
        def get_speakers_intervals(df,group_max_time_diff_ms,min_interval_time_ms,freq_ms):
                group_max_time_diff = np.timedelta64(group_max_time_diff_ms, 'ms')
                min_interval_time = np.timedelta64(min_interval_time_ms, 'ms')
                np_freq = np.timedelta64(freq_ms, 'ms')
                groups = groupwhile(df, lambda start,curr, next: df.index[next]-df.index[curr] <= group_max_time_diff)
                values = []
                for g in groups:
                    values.append([g.index[0], g.index[-1]+np_freq,len(g.index)]) # end time shoud include the end of the time interval
                stops = pd.DataFrame(values, columns=['start_time', 'end_time','cnt'])
                stops = stops[stops.end_time - stops.start_time >= min_interval_time]
                return stops

        speakers_intervals = get_speakers_intervals(df_speaker,group_max_time_diff_ms,min_interval_time_ms,freq_ms) # allow 3 timeslots of silence, interval should be at least 500 ms long
        return speakers_intervals

In [ ]:
df_speak_only = df_full[df_full.is_speak == 1]
speak_intervals = determine_speakers_intervals(df_speaker=df_speak_only,group_max_time_diff_ms=1000, min_interval_time_ms = 300,freq_ms=freq)

In [ ]:
speak_intervals[0:100]

In [ ]:
p2 = figure(title="Speaking intervals", y_axis_label='Speaker',x_axis_type="datetime",plot_width=1000,plot_height=300
            ,x_range=p1.x_range, tools=tools)#, y_range=p1.y_range)
x = []
y = []
x2 = []
y2 = []

for i in range(len(speak_intervals)):
    #print speakers_intervals.iloc[i]
    r = speak_intervals.iloc[i]
    x.append(r.start_time)
    y.append(1)
    x2.append(r.end_time)
    y2.append(1)

p2.segment(x, y, x2, y2,  
        line_width=15, line_alpha=1)

p = gridplot([[p1],[p2]])
show(p)

# export data

In [ ]:
df_fil.to_csv("E3_7E_44_77_F2_38_fil.csv", sep=',',index=False)

In [ ]:
df_full.to_csv("E3_7E_44_77_F2_38_fil_full.csv", sep=',',index=False)